In [1]:
import csv
import pandas as pd
import spacy
import nltk 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

In [2]:
import json

In [3]:
nlp = spacy.load('en_core_web_sm')
stops = stopwords.words('english')
stops.extend(["i'm", "he's", "i've", "i'll"])

In [4]:
def parse(rows):
    for row in rows:
        doc = nlp(row['Text'])
        row['tokens'] = []
        for token in doc:
            if token.pos_ != 'PUNCT' and token.is_stop is False:
                tokens = dict()
                tokens['word'] = token.text
                tokens['pos'] = token.pos_
                tokens['lemma'] = token.lemma_
                row['tokens'].append(tokens)
    return rows

In [5]:
def pos(rows):
    rows = parse(rows)
    for row in rows:
        pos_distribution = dict()
        for token in row['tokens']:
            if token['pos'] in pos_distribution.keys():
                pos_distribution[token['pos']] += 1
            else:
                pos_distribution[token['pos']] = 1
        row['pos'] = pos_distribution
    return rows

In [6]:
def phrase_len(rows):
    for row in rows:
        phrase = row['Text']
        words = [w.lower() for w in word_tokenize(phrase) if w.isalpha()]
        row['len'] = len(words)
    return rows

In [7]:
def word_order(sent):
    d = nlp(sent)
    roots = [token for token in d if token.head == token]
    orders = []
    for root in roots:
        d = {}
        d['V'] = root.i
        for child in root.children:
            if child.dep_ == 'nsubj':
                d['S'] = child.i
            if child.dep_== 'dobj':
                d['O'] = child.i
        listt = sorted(d.keys(),key=d.get)
        order = ''.join(listt)
        orders.append(order)
    return orders

In [8]:
def quotes_order(rows):
    for row in rows:
        row['word_order'] = word_order(row['Text'])
    return rows

In [9]:
punct_list = '!"«»“”#$%&\–-–—()*+,./\:;<=>?@[]^_`{|}~1234567890'

def clean(text):
    text = text.lower()
    for char in text:
        if char in punct_list:
            text = text.replace(char, '')
    return text

In [10]:
def popular_words(rows):
    txt_ps = []
    txt_pe = []
    txt_np = []
    
    populars = {'pricesses':{}, 'princes':{}, 'non-p':{}}

    for row in rows:
        # принцессы
        if row['Speaker_Status'] == 'PRINCESS':
            txt_ps.extend(extract_words(row['Text']))
            
        freq_ps = Counter(txt_ps)
        freq_psSorted = sorted(freq_ps.items(), key = lambda x: x[1], reverse=True)
        for i in freq_psSorted[:50]:
            populars['pricesses'][i[0]] = freq_ps[i[0]]

        # принцы
        if row['Speaker_Status'] == 'PRINCE':
            txt_pe.extend(extract_words(row['Text']))
            
            freq_pe = Counter(txt_pe)
            freq_peSorted = sorted(freq_pe.items(), key = lambda x: x[1], reverse=True)
            for i in freq_peSorted[:50]:
                populars['princes'][i[0]] = freq_ps[i[0]]

        # челядь
        if row['Speaker_Status'] == 'NON-P':
            txt_np.extend(extract_words(row['Text']))
            
            freq_np = Counter(txt_np)
            freq_npSorted = sorted(freq_np.items(), key = lambda x: x[1], reverse=True)
            for i in freq_npSorted[:50]:
                populars['non-p'][i[0]] = freq_ps[i[0]]
                
    return populars

In [12]:
rows = []

with open('princess_corpus.csv', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        row = dict(row)
        rows.append(row)

In [ ]:
data = pos(parse(rows))

In [ ]:
data = phrase_len(rows)

In [ ]:
data = quotes_order(rows)

In [ ]:
with open('new_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

In [18]:
# функция находящая полезные слова
def extract_words(text):
    words = clean(text).split()
    good_words = []
    for word in words:
        if word not in stops:
            good_words.append(word) 
    return good_words

In [ ]:
populars = popular_words(rows)

In [ ]:
print(populars)